In [1]:
!pip install texttable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install latextable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for latextable: filename=latextable-0.3.0-py3-none-any.whl size=7254 sha256=e5de8dd5a61531ef7b99f8f46215b55b80ac5f64de09af67fb328a2e2ffca0be
  Stored in directory: /root/.cache/pip/wheels/93/ec/52/9aa22d386bbfc8d6ade9be3f747b095443e0e452e6775b18ce
Successfully built latextable


In [3]:
pip install POT

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.3/682.3 KB 13.6 MB/s eta 0:00:00


In [17]:
import scipy.io as sio
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

from texttable import Texttable
import latextable

from scipy.spatial.distance import cdist
import ot

import warnings
warnings.filterwarnings('ignore')

In [40]:
file_names = [["caffeNet-dslr.mat", "caffeNet-webcam.mat"], ["googleNet-dslr.mat", "googleNet-webcam.mat"], 
     ["surf-dslr.mat", "surf-webcam.mat"]]
names = [["dslr", "webcam"], ["dslr", "webcam"], ["dslr", "webcam"]]

**Exercise 1: Subspace alignment**

In [35]:
def subspace_algn(S, T, d, Sy, Ty):

    """
    do a subspace alignmenet between S and T then return 1NN accuracy on
    T data.

    S: source domain data matrix
    T: target domain data matrix
    d: dimensionality reduction (n_components for PCA)
    Sy: source labels
    Ty: target labels

    """

    scaler = StandardScaler()

    S = scaler.fit_transform(S)
    T = scaler.fit_transform(T)

    principal = PCA(n_components=d)
    Xs = principal.fit_transform(S)
    Xs = principal.components_.T
    Xt = principal.fit_transform(T)
    Xt = principal.components_.T

    Xa = Xs@Xs.T@Xt
    Sa = S@Xa
    Ta = T@Xt

    knn = KNeighborsClassifier(n_neighbors=1).fit(Sa, np.ravel(Sy))
    preds = knn.predict(Ta)
        
    return accuracy_score(np.ravel(Ty), preds)


def k_1_NN(Source, Target, Slabels, Tlabels):

    """
    return 1NN accuracy on raw data without space alignment
    """
    
    knn = KNeighborsClassifier(n_neighbors=1).fit(Source, np.ravel(Slabels))
    preds = knn.predict(Target)
            
    return accuracy_score(np.ravel(Tlabels), preds)

def run_experiment_ex1(d=80):
    results = []
    for ij in files:
        i = ij[0]
        j = ij[1]

        #read data
        data = sio.loadmat(i)
        S = data['fts']
        Slabels = data['labels']

        data = sio.loadmat(j)
        T = data['fts']
        Tlabels = data['labels']

        # calculate source to target 1NN accuracy with and without applying subspace alignment
        s_t_acc_no_alignm = k_1_NN(Source=S, Target=T, Slabels=Slabels, Tlabels=Tlabels)
        s_t_acc_alignm = subspace_algn(S=S, T=T, d=d, Sy=Slabels, Ty=Tlabels)
        results.append([s_t_acc_no_alignm, s_t_acc_alignm])

        # calculate target to source 1NN accuracy with and without applying subspace alignment
        t_s_acc_no_alignm = k_1_NN(Source=T, Target=S, Slabels=Tlabels, Tlabels=Slabels)
        t_s_acc_alignm = subspace_algn(S=T, T=S, d=d, Sy=Tlabels, Ty=Slabels)
        results.append([t_s_acc_no_alignm, t_s_acc_alignm])

    return results

In [47]:
def print_experiment(results, d=None, reg=None):
    global files
    rows = [["Source", "Target", "1NN before", "1NN after"]]
    c=0
    for ij in names:
        i = ij[0]
        j = ij[1]
        row = [i, j, results[c][0], results[c][1]]
        rows.append(row)
        c+=1
        row = [j, i, results[c][0], results[c][1]]
        rows.append(row)
        c+=1

    table = Texttable()
    table.set_cols_align(["c"] * len(rows[0]))
    table.set_deco(Texttable.HEADER | Texttable.VLINES)

    table.add_rows(rows=rows)
    # print(table.draw())
    print()
    cpt = "1NN Accuracy Comparison Between Data from Different Domains in The Original Space (1NN before) and After Subspace Alignment (1NN after)"
    if d is not None:
        cpt+= f" (With d={d})."
    elif reg is not None:
        cpt+= f" (With reg={reg})"
    else:
        cpt+="."
    print(latextable.draw_latex(table, caption=cpt))

In [48]:
ds = [100, 80, 60, 30, 20, 10, 5, 2]
for d in ds:
    print_experiment(run_experiment_ex1(d=d), d=d)


\begin{table}
	\begin{center}
		\begin{tabular}{c|c|c|c}
			Source & Target & 1NN before & 1NN after \\
			\hline
			dslr & webcam & 0.963 & 0.993 \\
			webcam & dslr & 0.994 & 1 \\
			dslr & webcam & 0.997 & 0.990 \\
			webcam & dslr & 0.981 & 0.994 \\
			dslr & webcam & 0.441 & 0.800 \\
			webcam & dslr & 0.561 & 0.892 \\
		\end{tabular}
	\end{center}
	\caption{1NN Accuracy Comparison Between Data from Different Domains in The Original Space (1NN before) and After Subspace Alignment (1NN after) (With d=100).}
\end{table}

\begin{table}
	\begin{center}
		\begin{tabular}{c|c|c|c}
			Source & Target & 1NN before & 1NN after \\
			\hline
			dslr & webcam & 0.963 & 0.993 \\
			webcam & dslr & 0.994 & 1 \\
			dslr & webcam & 0.997 & 0.986 \\
			webcam & dslr & 0.981 & 0.994 \\
			dslr & webcam & 0.441 & 0.844 \\
			webcam & dslr & 0.561 & 0.917 \\
		\end{tabular}
	\end{center}
	\caption{1NN Accuracy Comparison Between Data from Different Domains in The Original Space (1NN before) and Afte

**Exercise 2: Entropic regularized optimal transport**

In [49]:
def  entropic_regularized(S, T, Slabels, Tlabels, reg):
    """
     Calculate entropic regularized OT problem between Source and Target and return
     1NN accuracy on the taget data.

     S: source data matrix
     T: target data matrix
     reg: regularization parameter
    """

    a = np.random.uniform(size=S.shape[0])
    b = np.random.uniform(size=T.shape[0])

    M = cdist(S, T, 'euclidean')
    M = M/M.max()

    coupling_mat = ot.sinkhorn(a, b, M, reg=0.1)

    Sa = coupling_mat@T

    knn = KNeighborsClassifier(n_neighbors=1).fit(Sa, np.ravel(Slabels))
    preds = knn.predict(T)
            
    return accuracy_score(np.ravel(Tlabels), preds)

def run_experiment_ex2(reg):
    results = []
    for ij in files:
        i = ij[0]
        j = ij[1]

        #read data
        data = sio.loadmat(i)
        S = data['fts']
        Slabels = data['labels']

        data = sio.loadmat(j)
        T = data['fts']
        Tlabels = data['labels']

        s_t_acc_no_alignm = k_1_NN(Source=S, Target=T, Slabels=Slabels, Tlabels=Tlabels)

        s_t_acc_alignm = entropic_regularized(S=S, T=T, Slabels=Slabels, Tlabels=Tlabels, reg=reg)

        #append source to target 1NN accuracy with and without applying subspace alignment
        results.append([s_t_acc_no_alignm, s_t_acc_alignm])

        t_s_acc_no_alignm = k_1_NN(Source=T, Target=S, Slabels=Tlabels, Tlabels=Slabels)
        t_s_acc_alignm = entropic_regularized(S=T, T=S, Slabels=Tlabels, Tlabels=Slabels, reg=reg)

        #append target to source 1NN accuracy with and without applying subspace alignment
        results.append([t_s_acc_no_alignm, t_s_acc_alignm])

    return results

In [50]:
regs = [0.25, 1, 10, 50, 100, 200, 500, 1000]
for reg in regs:
    print_experiment(run_experiment_ex2(reg=reg), reg=reg)


\begin{table}
	\begin{center}
		\begin{tabular}{c|c|c|c}
			Source & Target & 1NN before & 1NN after \\
			\hline
			dslr & webcam & 0.963 & 0.885 \\
			webcam & dslr & 0.994 & 0.987 \\
			dslr & webcam & 0.997 & 0.980 \\
			webcam & dslr & 0.981 & 0.987 \\
			dslr & webcam & 0.441 & 0.505 \\
			webcam & dslr & 0.561 & 0.554 \\
		\end{tabular}
	\end{center}
	\caption{1NN Accuracy Comparison Between Data from Different Domains in The Original Space (1NN before) and After Subspace Alignment (1NN after) (With reg=0.25)}
\end{table}

\begin{table}
	\begin{center}
		\begin{tabular}{c|c|c|c}
			Source & Target & 1NN before & 1NN after \\
			\hline
			dslr & webcam & 0.963 & 0.966 \\
			webcam & dslr & 0.994 & 0.987 \\
			dslr & webcam & 0.997 & 0.983 \\
			webcam & dslr & 0.981 & 0.994 \\
			dslr & webcam & 0.441 & 0.420 \\
			webcam & dslr & 0.561 & 0.529 \\
		\end{tabular}
	\end{center}
	\caption{1NN Accuracy Comparison Between Data from Different Domains in The Original Space (1NN before